In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
import pandas as pd
from torch.optim.lr_scheduler import CosineAnnealingLR
import copy
import time 
import torch.nn.functional as F

# Set random seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set default tensor type to float32
torch.set_default_tensor_type(torch.FloatTensor)

# Normalization

In [ ]:
class Normalizer:
    def __init__(self, x_min, x_max):
        self.x_min = x_min
        self.x_max = x_max
        
    def normalize(self, x):
        return 2.0 * (x - self.x_min) / (self.x_max - self.x_min) - 1.0
        
    def denormalize(self, x_norm):
        return 0.5 * (x_norm + 1.0) * (self.x_max - self.x_min) + self.x_min

# ANN

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden1 = nn.Linear(2, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.hidden3 = nn.Linear(128, 128)
        self.hidden4 = nn.Linear(128, 128)  
        self.hidden5 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 2)
    def forward(self, x):
        x = torch.tanh(self.hidden1(x))
        x = torch.tanh(self.hidden2(x))
        x = torch.tanh(self.hidden3(x))
        x = torch.tanh(self.hidden4(x))
        x = torch.tanh(self.hidden5(x))
        return self.output(x)  # return [u, v]
    # Define the problem domain using the given vertices

# Vertice

In [ ]:
vertices = np.array([
    [0, 0],
    [0, 4],
    [5, 4],
    [5, 0],
    [0, 0], # Closing the polygon
],dtype=np.float32)
path = Path(vertices)
def in_domain(x, y):
    points = np.column_stack((x.cpu().numpy(), y.cpu().numpy()))
    return torch.tensor(path.contains_points(points), dtype=torch.bool, device=device)

# BC

In [ ]:
# Define boundary conditions with tolerance
def BC_bottom(x, y):
    tol = 1e-6
    return ((torch.abs(y - 0) < tol) & (x >= 0) & (x <= 5)).squeeze()

def BC_left(x, y):
    tol = 1e-6
    return ((torch.abs(x - 0) < tol) & (y >= 0) & (y <= 4)).squeeze()

def BC_top(x, y):
    tol = 1e-6
    return ((torch.abs(y - 4) < tol) & (x >= 0) & (x <= 5)).squeeze()

def BC_right(x, y):
    tol = 1e-6
    return ((torch.abs(x - 5) < tol) & (y >= 0) & (y <= 4)).squeeze()


# Huber loss

In [ ]:
class HuberLoss:
    def __init__(self, delta=1.0):
        self.delta = delta
        
    def __call__(self, y_true, y_pred):
        error = torch.abs(y_true - y_pred)
        quadratic = torch.min(error, torch.tensor(self.delta))
        linear = error - quadratic
        return torch.mean(0.5 * quadratic ** 2 + self.delta * linear)

# BC Loss

In [ ]:
def BC(xy, net):
    x, y = xy[:, 0].unsqueeze(1), xy[:, 1].unsqueeze(1)
    
    output = net(xy)
    u = output[:, 0:1]  # Split output channel (u)
    v = output[:, 1:2]  # Split output channel (v)
    # Get boundary conditions
    
    bc_b = BC_bottom(x, y)
    bc_l = BC_left(x, y)
    bc_t = BC_top(x, y)
    bc_r = BC_right(x, y)
    
    huber = HuberLoss(delta=1.0)
    
    loss = huber(u[bc_b], torch.zeros_like(u[bc_b])) + huber(v[bc_b], torch.zeros_like(v[bc_b])) # ux = 0, uy = 0 at bottom
    loss += huber(u[bc_l], torch.zeros_like(u[bc_l])) # ux = 0 at left
    loss += huber(u[bc_r], torch.zeros_like(u[bc_r])) # ux = 0 at right
    
    # Process top boundary
    xy_top = xy[bc_t].requires_grad_(True)
    x_top = xy_top[:, 0:1]  # x-coordinates of points on the top boundary
    
    output_top = net(xy_top)
    u_top = output_top[:, 0:1]
    v_top = output_top[:, 1:2]
    
    u_x_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 0]
    u_y_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 1]
    v_x_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 0]
    v_y_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 1]
    
    E = 15   # Young's modulus MPa
    nu = 0.3  # Poisson's ratio
    
    # Calculate stresses
    sigma_xx_top = (E / ((1 + nu) * (1 - 2 * nu))) * ((1 - nu) * u_x_top + nu * v_y_top)
    sigma_yy_top = (E / ((1 + nu) * (1 - 2 * nu))) * (nu * u_x_top + (1 - nu) * v_y_top)
    sigma_xy_top = E / (2 * (1 + nu)) * (u_y_top + v_x_top)
    
    # Create mask for points with distributed load (x from 0 to 1)
    load_mask = (x_top >= 2) & (x_top <= 3)
    load_mask = load_mask.squeeze()  # Remove extra dimension
    
    # All points on top have sigma_xy = 0
    loss += huber(sigma_xy_top, torch.zeros_like(sigma_xy_top))
    
    # Add condition for the area without load on the top edge
    no_load_mask = ~load_mask
    if torch.any(no_load_mask):
 
        loss += huber(sigma_yy_top[no_load_mask], torch.zeros_like(sigma_yy_top[no_load_mask]))
    
    # Points with load have sigma_yy = -1.5 (negative for compression)
    if torch.any(load_mask):
        loss += huber(sigma_yy_top[load_mask], -0.5 * torch.ones_like(sigma_yy_top[load_mask]))
    
    # Process right boundary
    xy_right = xy[bc_r].requires_grad_(True)
    output_right = net(xy_right)
    u_right = output_right[:, 0:1]
    v_right = output_right[:, 1:2]
    
    u_y_right = torch.autograd.grad(u_right.sum(), xy_right, create_graph=True)[0][:, 1]
    v_x_right = torch.autograd.grad(v_right.sum(), xy_right, create_graph=True)[0][:, 0]
    u_x_right = torch.autograd.grad(u_right.sum(), xy_right, create_graph=True)[0][:, 0]
    v_y_right = torch.autograd.grad(v_right.sum(), xy_right, create_graph=True)[0][:, 1]
    sigma_xy_right = E / (2 * (1 + nu)) * (u_y_right + v_x_right)
    sigma_xx_right = (E / ((1 + nu) * (1 - 2 * nu))) * ((1 - nu) * u_x_right + nu * v_y_right)
    loss += huber(sigma_xy_right, torch.zeros_like(sigma_xy_right)) # Sigma_xy = 0 at right
    loss += huber(sigma_xx_right, torch.zeros_like(sigma_xx_right)) # Sigma_xy = 0 at right
    # Process left boundary
    xy_left = xy[bc_l].requires_grad_(True)
    output_left = net(xy_left)
    u_left = output_left[:, 0:1]
    v_left = output_left[:, 1:2]
    
    u_y_left = torch.autograd.grad(u_left.sum(), xy_left, create_graph=True)[0][:, 1]
    v_x_left = torch.autograd.grad(v_left.sum(), xy_left, create_graph=True)[0][:, 0]
    
    sigma_xy_left = E / (2 * (1 + nu)) * (u_y_left + v_x_left)
    loss += huber(sigma_xy_left, torch.zeros_like(sigma_xy_left)) # Sigma_xy = 0 at left

    # Process bottom boundary
    xy_bottom = xy[bc_b].requires_grad_(True)
    output_bottom = net(xy_bottom)
    u_bottom = output_bottom[:, 0:1]
    v_bottom = output_bottom[:, 1:2]
    
    u_y_bottom = torch.autograd.grad(u_bottom.sum(), xy_bottom, create_graph=True)[0][:, 1]
    v_x_bottom = torch.autograd.grad(v_bottom.sum(), xy_bottom, create_graph=True)[0][:, 0]
    u_x_bottom = torch.autograd.grad(u_bottom.sum(), xy_bottom, create_graph=True)[0][:, 0]
    v_y_bottom = torch.autograd.grad(v_bottom.sum(), xy_bottom, create_graph=True)[0][:, 1]
    sigma_xy_bottom = E / (2 * (1 + nu)) * (u_y_bottom + v_x_bottom)
    sigma_xx_bottom = (E / ((1 + nu) * (1 - 2 * nu))) * ((1 - nu) * u_x_bottom + nu * v_y_bottom)
    sigma_yy_bottom = (E / ((1 + nu) * (1 - 2 * nu))) * (nu * u_x_bottom + (1 - nu) * v_y_bottom)
    loss += huber(sigma_xy_bottom, torch.zeros_like(sigma_xy_bottom)) # Sigma_xy = 0 at bottom
    loss += huber(sigma_xx_bottom, torch.zeros_like(sigma_xx_bottom)) # Sigma_xx = 0 at bottom
    loss += huber(sigma_yy_bottom, torch.zeros_like(sigma_yy_bottom)) # Sigma_xx = 0 at bottom
    return loss

In [ ]:
def train(net, optimizer, n_epochs, fem_data_file=None, nx=100, ny=100):
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=1e-6)
    
    best_loss = float('inf')
    patience = 0
    max_patience = 3000
    
    prev_net = None
    
    #Add lists for loss history
    loss_history = []
    pde_x_history = []
    pde_y_history = []
    bc_history = []
    epoch_history = []
    
    # Create training points only once
    print("Generating training points...")
    x_inner, y_inner = generate_point_distribution(nx=nx, ny=ny)
    print(f"Generated {len(x_inner)} interior points")
    
    # Create boundary points only once
    print("Generating boundary points...")
    num_boundary_points = 100 # Number of points on each boundary
    
    # Create points on the bottom edge (y=0)
    x_bottom = torch.linspace(0, 5, num_boundary_points, device=device)
    y_bottom = torch.zeros(num_boundary_points, device=device)
    
    # Create points on the left edge (x=0)
    x_left = torch.zeros(num_boundary_points, device=device)
    y_left = torch.linspace(0, 4, num_boundary_points, device=device)
    
    # Create points on the top edge (y=4)
    x_top = torch.linspace(0, 5, num_boundary_points, device=device)
    y_top = torch.ones(num_boundary_points, device=device) * 4
    
    # Create points on the right edge (x=5)
    x_right = torch.ones(num_boundary_points, device=device) * 5
    y_right = torch.linspace(0, 4, num_boundary_points, device=device)
    
    # Total boundary points
    x_b = torch.cat([x_bottom, x_left, x_top, x_right])
    y_b = torch.cat([y_bottom, y_left, y_top, y_right])
    
    # Create tensor for BC function
    xy_b = torch.stack([x_b, y_b], dim=1)
    print(f"Generated {len(xy_b)} boundary points")
    
    print(f"Starting training for {n_epochs} epochs...")
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        
        # using x_inner, y_inner
        loss_pde_x, loss_pde_y = PDE(x_inner, y_inner, net)
        loss_bc = BC(xy_b, net)
        
        # calculate total loss
        loss = loss_pde_x + loss_pde_y + loss_bc
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=0.5)
        
        optimizer.step()
        scheduler.step()
        
        # Save history
        loss_history.append(loss.item())
        pde_x_history.append(loss_pde_x.item())
        pde_y_history.append(loss_pde_y.item())
        bc_history.append(loss_bc.item())
        epoch_history.append(epoch)
        
        if loss < best_loss:
            best_loss = loss
            patience = 0
            
            prev_net = copy.deepcopy(net)
            
            torch.save({
                'net_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': loss,
                'epoch': epoch,
            }, 'best_model.pth')
        else:
            patience += 1
            if patience > max_patience:
                print(f"Early stopping at epoch {epoch}")
                break
                
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.10f}, '
                  f'PDE_x: {loss_pde_x.item():.10f}, PDE_y: {loss_pde_y.item():.10f}, '
                  f'BC: {loss_bc.item():.10f}, LR: {scheduler.get_last_lr()[0]:.10f}')
    
    # Creat graph Loss history
    plt.figure(figsize=(12, 8))
    
    # Plot total loss
    plt.subplot(2, 1, 1)
    plt.plot(epoch_history, loss_history, 'b-', label='Total Loss')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (log scale)')
    plt.title('Training Loss History')
    plt.legend()
    plt.grid(True)
    
    # Plot component losses
    plt.subplot(2, 1, 2)
    plt.plot(epoch_history, pde_x_history, 'r-', label='PDE_x Loss')
    plt.plot(epoch_history, pde_y_history, 'g-', label='PDE_y Loss')
    plt.plot(epoch_history, bc_history, 'b-', label='BC Loss')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Component Losses (log scale)')
    plt.title('Component Losses History')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('loss_history.png')
    plt.close()
    
    #additional return value for metrics for tracking training progress
    metrics = {
        'loss_history': loss_history,
        'pde_x_history': pde_x_history,
        'pde_y_history': pde_y_history,
        'bc_history': bc_history,
        'epoch_history': epoch_history
    }
    
    print("Training completed.")
    return net, metrics

# Datapoint sampling

In [ ]:
def generate_point_distribution(nx=100, ny=100, load_density=10, load_influence=70):
    # Defined boundaries of the domain
    x_min, x_max = 0.0, 5.0
    y_min, y_max = 0.0, 4.0
    
    # Create random points in the domain
    # Add number of points to be 4 times the grid size
    num_random_points = nx * ny * 4
    x_random = torch.rand(num_random_points, device=device) * (x_max - x_min) + x_min
    y_random = torch.rand(num_random_points, device=device) * (y_max - y_min) + y_min
    
    print(f"Total random points generated: {len(x_random)}")
    
    load_x_start, load_x_end = 2.0, 3.0
    load_y = 4.0
    
    distances = torch.zeros_like(x_random)
    
   
    in_x_range = (x_random >= load_x_start) & (x_random <= load_x_end)
    
    distances[in_x_range] = torch.abs(y_random[in_x_range] - load_y)
    
     
    left_of_range = x_random < load_x_start
    
    distances[left_of_range] = torch.sqrt(
        (x_random[left_of_range] - load_x_start)**2 + 
        (y_random[left_of_range] - load_y)**2
    )
    
    
    right_of_range = x_random > load_x_end
    
    distances[right_of_range] = torch.sqrt(
        (x_random[right_of_range] - load_x_end)**2 + 
        (y_random[right_of_range] - load_y)**2
    )
    
    base_prob = 0.2 
    additional_prob = load_influence / (1 + (distances * load_density)**2)
    
    
    keep_prob = torch.clamp(base_prob + additional_prob, 0, 1)
    
    
    keep_mask = torch.rand(num_random_points, device=device) < keep_prob
    
    
    x_final = x_random[keep_mask]
    y_final = y_random[keep_mask]
    
    print(f"Final points after density-based filtering: {len(x_final)}")
    
    
    plt.figure(figsize=(10, 8))
    
    
    plt.scatter(x_final.cpu().numpy(), y_final.cpu().numpy(), s=1, color='blue', alpha=0.7)
    
    
    plt.plot([load_x_start, load_x_end], [load_y, load_y], 'r-', linewidth=2, label='Load line')
    
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Point Distribution with Enhanced Load Area Density - {len(x_final)} points')
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.gca().set_aspect('equal')
    plt.legend()
    
    plt.show()
    
    return x_final, y_final


x_points, y_points = generate_point_distribution(nx=100, ny=100, load_density=10, load_influence=70)

# PDE

In [ ]:
def PDE(x, y, net):
    # Reshape 1D tensors into column vectors for concatenation
    x_reshaped = x.unsqueeze(1) if x.dim() == 1 else x
    y_reshaped = y.unsqueeze(1) if y.dim() == 1 else y
    
    xy = torch.cat([x_reshaped, y_reshaped], dim=1)
    xy.requires_grad = True
    
    output = net(xy)
    u = output[:, 0:1]
    v = output[:, 1:2]
    
    u_x = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    u_y = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    v_x = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    v_y = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    
    E = 15  # Young's modulus MPa 
    nu = 0.3 # Poisson's ratio
    gamma = 0.018 # Distributed load (Mn/m^2)
    sigma_xx = E / (1 - nu**2) * (u_x + nu * v_y)
    sigma_yy = E / (1 - nu**2) * (v_y + nu * u_x)
    sigma_xy = E / (2 * (1 + nu)) * (u_y + v_x)
    
    f_x = torch.zeros_like(x_reshaped)
    f_y = -gamma * torch.ones_like(y_reshaped)
    
    R_x = torch.autograd.grad(sigma_xx.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_xy.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_x
    R_y = torch.autograd.grad(sigma_xy.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_yy.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_y
    
    huber = HuberLoss(delta=1.0)
    loss_x = huber(R_x, torch.zeros_like(R_x))
    loss_y = huber(R_y, torch.zeros_like(R_y))
    
    return loss_x, loss_y

# Training 


In [ ]:
def train(net, optimizer, n_epochs, fem_data_file=None, nx=100, ny=100):
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=1e-6)
    
    best_loss = float('inf')
    patience = 0
    max_patience = 3000
    
    prev_net = None
    
    
    loss_history = []
    pde_x_history = []
    pde_y_history = []
    bc_history = []
    epoch_history = []
    
    
    print("Generating training points...")
    x_inner, y_inner = generate_point_distribution(nx=nx, ny=ny)
    print(f"Generated {len(x_inner)} interior points")
    
    
    print("Generating boundary points...")
    num_boundary_points = 1000 
    
    
    x_bottom = torch.linspace(0, 5, num_boundary_points, device=device)
    y_bottom = torch.zeros(num_boundary_points, device=device)
    
    
    x_left = torch.zeros(num_boundary_points, device=device)
    y_left = torch.linspace(0, 4, num_boundary_points, device=device)
    
    
    x_top = torch.linspace(0, 5, num_boundary_points, device=device)
    y_top = torch.ones(num_boundary_points, device=device) * 4
    
    
    x_right = torch.ones(num_boundary_points, device=device) * 5
    y_right = torch.linspace(0, 4, num_boundary_points, device=device)
    
    
    x_b = torch.cat([x_bottom, x_left, x_top, x_right])
    y_b = torch.cat([y_bottom, y_left, y_top, y_right])
    
    
    xy_b = torch.stack([x_b, y_b], dim=1)
    print(f"Generated {len(xy_b)} boundary points")
    
    print(f"Starting training for {n_epochs} epochs...")
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        
        
        loss_pde_x, loss_pde_y = PDE(x_inner, y_inner, net)
        loss_bc = BC(xy_b, net)
        
        
        loss = loss_pde_x + loss_pde_y + loss_bc
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=0.5)
        
        optimizer.step()
        scheduler.step()
        
       
        loss_history.append(loss.item())
        pde_x_history.append(loss_pde_x.item())
        pde_y_history.append(loss_pde_y.item())
        bc_history.append(loss_bc.item())
        epoch_history.append(epoch)
        
        if loss < best_loss:
            best_loss = loss
            patience = 0
            
            prev_net = copy.deepcopy(net)
            
            torch.save({
                'net_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': loss,
                'epoch': epoch,
            }, 'best_model.pth')
        else:
            patience += 1
            if patience > max_patience:
                print(f"Early stopping at epoch {epoch}")
                break
                
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.10f}, '
                  f'PDE_x: {loss_pde_x.item():.10f}, PDE_y: {loss_pde_y.item():.10f}, '
                  f'BC: {loss_bc.item():.10f}, LR: {scheduler.get_last_lr()[0]:.10f}')
    
    
    plt.figure(figsize=(12, 8))
    
    
    plt.subplot(2, 1, 1)
    plt.plot(epoch_history, loss_history, 'b-', label='Total Loss')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (log scale)')
    plt.title('Training Loss History')
    plt.legend()
    plt.grid(True)
    
    
    plt.subplot(2, 1, 2)
    plt.plot(epoch_history, pde_x_history, 'r-', label='PDE_x Loss')
    plt.plot(epoch_history, pde_y_history, 'g-', label='PDE_y Loss')
    plt.plot(epoch_history, bc_history, 'b-', label='BC Loss')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Component Losses (log scale)')
    plt.title('Component Losses History')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('loss_history.png')
    plt.close()
    
   
    metrics = {
        'loss_history': loss_history,
        'pde_x_history': pde_x_history,
        'pde_y_history': pde_y_history,
        'bc_history': bc_history,
        'epoch_history': epoch_history
    }
    
    print("Training completed.")
    return net, metrics

# Plot result

In [ ]:
def plot_results(net, output_filename='Footing_data.csv'):
    nx, ny = 100, 100
    X = torch.linspace(0, 5.0, nx, device=device)  
    Y = torch.linspace(0, 4.0, ny, device=device)  
    
    xx, yy = torch.meshgrid(X, Y, indexing='ij')
    
    X = xx.flatten()
    Y = yy.flatten()
    
   
    mask = in_domain(X, Y)
    mask_np = mask.cpu().numpy()  
    
    
    XY = torch.stack([X, Y], dim=1)
    XY.requires_grad_(True)
    
    with torch.enable_grad():
        output = net(XY)
        U = output[:, 0:1]  
        V = output[:, 1:2]  
        
        U_x = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 0]
        U_y = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 1]
        V_x = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 0]
        V_y = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 1]
        
        E = 15  # Mpa Young's modulus
        nu = 0.3  # Poisson's ratio
        
        sigma_xx = (E / ((1 + nu) * (1 - 2 * nu))) * ((1 - nu) * U_x + nu * V_y)
        sigma_yy = (E / ((1 + nu) * (1 - 2 * nu))) * (nu * U_x + (1 - nu) * V_y)
        sigma_xy = E / (2 * (1 + nu)) * (U_y + V_x)
    
    
    X_np = X.detach().cpu().numpy()
    Y_np = Y.detach().cpu().numpy()
    U_full = U.detach().cpu().numpy().squeeze()
    V_full = V.detach().cpu().numpy().squeeze()
    sigma_xx_full = sigma_xx.detach().cpu().numpy().squeeze()*1000
    sigma_yy_full = sigma_yy.detach().cpu().numpy().squeeze()*1000
    sigma_xy_full = sigma_xy.detach().cpu().numpy().squeeze()*1000
    
    
    magnitude = np.sqrt(U_full**2 + V_full**2)
    
    
    df_out = pd.DataFrame({
        'X': X_np[mask_np],
        'Y': Y_np[mask_np],
        'ux': U_full[mask_np],
        'uy': V_full[mask_np],
        'sigma_xx': sigma_xx_full[mask_np],
        'sigma_yy': sigma_yy_full[mask_np],
        'sigma_xy': sigma_xy_full[mask_np],
        'magnitude': magnitude[mask_np]
    })
    
    
    df_out.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")
    
    
    mask_cpu = mask.cpu().numpy()
    U_masked = np.ma.masked_array(U_full, mask=~mask_cpu)
    V_masked = np.ma.masked_array(V_full, mask=~mask_cpu)
    sigma_xx_masked = np.ma.masked_array(sigma_xx_full, mask=~mask_cpu)
    sigma_yy_masked = np.ma.masked_array(sigma_yy_full, mask=~mask_cpu)
    sigma_xy_masked = np.ma.masked_array(sigma_xy_full, mask=~mask_cpu)
    magnitude_masked = np.ma.masked_array(magnitude, mask=~mask_cpu)
    
    
    plt.figure(figsize=(16, 12))
    
    
    plt.subplot(2, 3, 1)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), U_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Displacement ux (m)')
    plt.title('Displacement in x-direction')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    
    plt.subplot(2, 3, 2)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), V_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Displacement uy (m)')
    plt.title('Displacement in y-direction')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    
    plt.subplot(2, 3, 3)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), magnitude_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Displacement magnitude (m)')
    plt.title('Displacement magnitude')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    
    plt.subplot(2, 3, 4)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), sigma_xx_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Stress σxx (kPa)')
    plt.title('Normal stress σxx')
    plt.xlabel('X')
    plt.ylabel('Y')
    
   
    plt.subplot(2, 3, 5)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), sigma_yy_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Stress σyy (kPa)')
    plt.title('Normal stress σyy')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    
    plt.subplot(2, 3, 6)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), sigma_xy_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Stress σxy (kPa)')
    plt.title('Shear stress σxy')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    plt.tight_layout()
    plt.savefig('results_plots.png', dpi=300)
    plt.close()
    print("Result plots saved to results_plots.png")

In [ ]:
# Main execution
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Initialize network and optimizer
    net = Net().to(device)
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    
    # Train the network
    trained_net, metrics = train(net=net, optimizer=optimizer, n_epochs=4000)
    # Plot results
    try:
        plot_results(trained_net)
    except Exception as e:
        print(f"Error in plot_results: {str(e)}")